In [2]:
import pandas as pd

java = pd.read_csv('./java/java_mb_KG.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
java

,user_id,item_id,behavior,timestamp
0,1,305,watch,1406863244
1,1,172,done,1410874261
2,1,54,done,1410874265
3,1,54,duration,1410874275
4,1,314,done,1410913417
...,...,...,...,...
2489,183,294,done,1429083169
2490,183,294,tag,1429083255
2491,183,294,duration,1429083393
2492,183,357,done,1429084813


In [6]:
java.groupby('user_id')['timestamp'].mean().reset_index()

,user_id,timestamp
0,1,1.419819e+09
1,2,1.422211e+09
2,3,1.417223e+09
3,4,1.421624e+09
4,5,1.417521e+09
...,...,...
178,179,1.427677e+09
179,180,1.411322e+09
180,181,1.418450e+09
181,182,1.410552e+09


In [7]:
java_user_count = java.groupby('user_id').count().reset_index()[['user_id','item_id']]
java_inter = java[java['user_id'].isin(java_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()
java_inter

,user_id,item_id
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310..."
1,2,"[363, 1]"
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29..."
3,4,"[363, 1]"
4,5,"[1, 294, 363]"
...,...,...
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]"
179,180,[1]
180,181,"[363, 1]"
181,182,"[294, 54, 1]"


In [8]:
java_inter['time_avg'] = java.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
java_inter['time_var'] = java.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
java_inter['time_max'] = java.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
java_inter['time_min'] = java.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
java_inter['inter_len'] = java_inter['item_id'].apply(lambda x : len(x))
java_inter = java_inter[java_inter['inter_len']>=3]
java_inter = java_inter.fillna(0)
java_inter

,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",1.419819e+09,6.272646e+13,1429618603,1406863244,33
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",1.417223e+09,4.107035e+12,1419596727,1414167044,11
4,5,"[1, 294, 363]",1.417521e+09,2.370145e+13,1427458885,1415533240,3
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",1.419823e+09,1.068651e+13,1424172965,1415181904,33
6,7,"[294, 1, 363]",1.416762e+09,1.172146e+12,1417246874,1414824787,3
...,...,...,...,...,...,...,...
176,177,"[1, 337, 294]",1.428904e+09,9.838829e+05,1428905155,1428902727,3
177,178,"[298, 311, 1, 331]",1.426928e+09,1.063703e+05,1426928922,1426928225,4
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",1.427677e+09,2.098206e+10,1427968846,1427595001,8
181,182,"[294, 54, 1]",1.410552e+09,1.452166e+11,1411123740,1410361547,3


In [9]:
from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = java_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['scene'] = predict_y
java_inter

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len,scene
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",1.419819e+09,6.272646e+13,1429618603,1406863244,33,3
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",1.417223e+09,4.107035e+12,1419596727,1414167044,11,0
4,5,"[1, 294, 363]",1.417521e+09,2.370145e+13,1427458885,1415533240,3,3
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",1.419823e+09,1.068651e+13,1424172965,1415181904,33,0
6,7,"[294, 1, 363]",1.416762e+09,1.172146e+12,1417246874,1414824787,3,0
...,...,...,...,...,...,...,...,...
176,177,"[1, 337, 294]",1.428904e+09,9.838829e+05,1428905155,1428902727,3,1
177,178,"[298, 311, 1, 331]",1.426928e+09,1.063703e+05,1426928922,1426928225,4,1
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",1.427677e+09,2.098206e+10,1427968846,1427595001,8,1
181,182,"[294, 54, 1]",1.410552e+09,1.452166e+11,1411123740,1410361547,3,0


In [62]:
java_inter[java_inter['user_id']==62]

,user_id,item_id,done_count,watch_count,enter_count,tag_count,dur_count,inter_len,cat
61,62,"[294, 1, 295, 35, 297, 300, 301, 306, 307, 311...",0.0,0.0,1.0,0.0,0.0,45,1


In [10]:
alldata = []
for user in java_inter['user_id'].tolist():
    # print(user)
    seq_len = len(java_inter[java_inter['user_id']==user]['item_id'].item())
    total_seq = java_inter[java_inter['user_id']==1]['item_id'].item().tolist()
    scene = java_inter[java_inter['user_id']==user]['scene'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
java_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','target','pred_path'])
java_new

,user_id,inter_seq,scene,target,pred_path
0,1,[305],3,172,[172]
1,1,[305],3,54,"[172, 54]"
2,1,[305],3,314,"[172, 54, 314]"
3,1,[305],3,1,"[172, 54, 314, 1]"
4,1,[305],3,350,"[172, 54, 314, 1, 350]"
...,...,...,...,...,...
9854,183,[305],3,54,"[172, 54]"
9855,183,[305],3,314,"[172, 54, 314]"
9856,183,"[305, 172]",3,54,[54]
9857,183,"[305, 172]",3,314,"[54, 314]"


In [11]:
java_new[java_new['pred_path'].apply(lambda x: len(x))<=5]

,user_id,inter_seq,scene,target,pred_path
0,1,[305],3,172,[172]
1,1,[305],3,54,"[172, 54]"
2,1,[305],3,314,"[172, 54, 314]"
3,1,[305],3,1,"[172, 54, 314, 1]"
4,1,[305],3,350,"[172, 54, 314, 1, 350]"
...,...,...,...,...,...
9854,183,[305],3,54,"[172, 54]"
9855,183,[305],3,314,"[172, 54, 314]"
9856,183,"[305, 172]",3,54,[54]
9857,183,"[305, 172]",3,314,"[54, 314]"


In [12]:
java_test = java_new[java_new['pred_path'].apply(lambda x: len(x))<=5].sample(frac=0.25)
java_train = java_new[~java_new.index.isin(java_test)]
java_test

,user_id,inter_seq,scene,target,pred_path
4685,75,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",0,319,"[337, 319]"
9773,163,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",1,343,"[341, 343]"
3528,67,[305],1,54,"[172, 54]"
8086,107,"[305, 172, 54, 314, 1, 350]",0,310,"[352, 301, 302, 310]"
606,6,"[305, 172, 54]",0,352,"[314, 1, 350, 352]"
...,...,...,...,...,...
4960,79,"[305, 172, 54, 314]",0,301,"[1, 350, 352, 301]"
9396,158,"[305, 172, 54, 314, 1]",0,310,"[350, 352, 301, 302, 310]"
1358,21,"[305, 172, 54, 314, 1, 350]",1,302,"[352, 301, 302]"
2509,62,"[305, 172, 54, 314, 1, 350, 352, 301]",0,337,"[302, 310, 312, 337]"


In [13]:
java_test.to_csv('java/java_test.csv',header=True,sep='\t',index=False)
java_train.to_csv('java/java_train.csv',header=True,sep='\t',index=False)

In [ ]:
# 保持两方法测试集和训练集一致

In [14]:
beh_count = java.groupby(['user_id','behavior']).count().reset_index()
java_inter['done_count'] = beh_count[beh_count['behavior']=='done']['item_id']
java_inter['watch_count'] = beh_count[beh_count['behavior']=='watch']['item_id']
java_inter['enter_count'] = beh_count[beh_count['behavior']=='enter']['item_id']
java_inter['tag_count'] = beh_count[beh_count['behavior']=='tag']['item_id']
java_inter['dur_count'] = beh_count[beh_count['behavior']=='dur']['item_id']
java_inter['time_avg'] = java.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
java_inter['time_var'] = java.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
java_inter['time_max'] = java.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
java_inter['time_min'] = java.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
java_inter['inter_len'] = java_inter['item_id'].apply(lambda x : len(x))
java_inter = java_inter[java_inter['inter_len']>=3]
java_inter = java_inter.fillna(0)
java_inter

,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len,scene,done_count,watch_count,enter_count,tag_count,dur_count
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",1.419819e+09,6.272646e+13,1429618603,1406863244,33,3,31.0,0.0,0.0,0.0,0.0
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",1.417223e+09,4.107035e+12,1419596727,1414167044,11,0,0.0,0.0,0.0,6.0,0.0
4,5,"[1, 294, 363]",1.417521e+09,2.370145e+13,1427458885,1415533240,3,3,1.0,0.0,0.0,0.0,0.0
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",1.419823e+09,1.068651e+13,1424172965,1415181904,33,0,0.0,0.0,1.0,0.0,0.0
6,7,"[294, 1, 363]",1.416762e+09,1.172146e+12,1417246874,1414824787,3,0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,177,"[1, 337, 294]",1.428904e+09,9.838829e+05,1428905155,1428902727,3,1,10.0,0.0,0.0,0.0,0.0
177,178,"[298, 311, 1, 331]",1.426928e+09,1.063703e+05,1426928922,1426928225,4,1,0.0,0.0,0.0,0.0,0.0
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",1.427677e+09,2.098206e+10,1427968846,1427595001,8,1,0.0,0.0,0.0,11.0,0.0
181,182,"[294, 54, 1]",1.410552e+09,1.452166e+11,1411123740,1410361547,3,0,0.0,0.0,0.0,0.0,0.0


In [15]:
from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=5)
train_x = java_inter[['done_count','watch_count','enter_count','tag_count','dur_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['cat'] = predict_y
java_inter

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len,scene,done_count,watch_count,enter_count,tag_count,dur_count,cat
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",1.419819e+09,6.272646e+13,1429618603,1406863244,33,3,31.0,0.0,0.0,0.0,0.0,4
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",1.417223e+09,4.107035e+12,1419596727,1414167044,11,0,0.0,0.0,0.0,6.0,0.0,3
4,5,"[1, 294, 363]",1.417521e+09,2.370145e+13,1427458885,1415533240,3,3,1.0,0.0,0.0,0.0,0.0,2
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",1.419823e+09,1.068651e+13,1424172965,1415181904,33,0,0.0,0.0,1.0,0.0,0.0,1
6,7,"[294, 1, 363]",1.416762e+09,1.172146e+12,1417246874,1414824787,3,0,0.0,0.0,0.0,1.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,177,"[1, 337, 294]",1.428904e+09,9.838829e+05,1428905155,1428902727,3,1,10.0,0.0,0.0,0.0,0.0,2
177,178,"[298, 311, 1, 331]",1.426928e+09,1.063703e+05,1426928922,1426928225,4,1,0.0,0.0,0.0,0.0,0.0,2
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",1.427677e+09,2.098206e+10,1427968846,1427595001,8,1,0.0,0.0,0.0,11.0,0.0,3
181,182,"[294, 54, 1]",1.410552e+09,1.452166e+11,1411123740,1410361547,3,0,0.0,0.0,0.0,0.0,0.0,2


In [16]:
alldata = []
for user in java_inter['user_id'].tolist():
    # print(user)
    seq_len = len(java_inter[java_inter['user_id']==user]['item_id'].item())
    total_seq = java_inter[java_inter['user_id']==user]['item_id'].item().tolist()
    scene = java_inter[java_inter['user_id']==user]['scene'].item()
    cat = java_inter[java_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
java_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
java_new

,user_id,inter_seq,scene,cat,target,pred_path
0,1,[305],3,4,172,[172]
1,1,[305],3,4,54,"[172, 54]"
2,1,[305],3,4,314,"[172, 54, 314]"
3,1,[305],3,4,1,"[172, 54, 314, 1]"
4,1,[305],3,4,350,"[172, 54, 314, 1, 350]"
...,...,...,...,...,...,...
9854,183,[305],3,1,54,"[172, 54]"
9855,183,[305],3,1,314,"[172, 54, 314]"
9856,183,"[305, 172]",3,1,54,[54]
9857,183,"[305, 172]",3,1,314,"[54, 314]"


In [18]:
java_test = java_new[java_new['pred_path'].apply(lambda x: len(x))<=5].sample(frac=0.25)
java_train = java_new[~java_new.index.isin(java_test)]
java_test

,user_id,inter_seq,scene,cat,target,pred_path
9047,137,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",1,2,311,"[337, 319, 311]"
1612,28,"[305, 172]",1,2,350,"[54, 314, 1, 350]"
6596,88,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",2,2,322,"[306, 173, 331, 322]"
8782,134,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",2,1,35,"[323, 300, 294, 295, 35]"
4734,76,"[305, 172, 54]",0,2,352,"[314, 1, 350, 352]"
...,...,...,...,...,...,...
2798,62,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",0,1,294,"[343, 321, 323, 300, 294]"
4230,71,"[305, 172, 54, 314, 1, 350, 352, 301]",0,0,312,"[302, 310, 312]"
3566,67,"[305, 172, 54, 314, 1, 350, 352]",1,2,301,[301]
1498,26,[305],1,2,172,[172]


In [19]:
java_test.to_csv('java/java_test_same.csv',header=True,sep='\t',index=False)
java_train.to_csv('java/java_train_same.csv',header=True,sep='\t',index=False)

In [1]:
import pandas as pd

net = pd.read_csv('./net/net_mb_KG.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
net_user_count = net.groupby('user_id').count().reset_index()[['user_id','item_id']]
net_inter = net[net['user_id'].isin(net_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()

net_inter['time_avg'] = net.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
net_inter['time_var'] = net.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
net_inter['time_max'] = net.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
net_inter['time_min'] = net.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
net_inter['inter_len'] = net_inter['item_id'].apply(lambda x : len(x))
net_inter = net_inter[net_inter['inter_len']>=3]
net_inter = net_inter.fillna(0)
net_inter

,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len
1,2,"[1, 297, 291, 286, 285, 290]",1.420932e+09,2.097459e+13,1425100667,1414155169,6
2,3,"[1, 285, 286, 287, 289, 290, 292, 294, 296, 29...",1.425333e+09,1.421498e+13,1426688331,1415095892,12
3,4,"[1, 285, 286, 287, 288, 289, 290, 291, 292, 29...",1.428229e+09,5.407589e+07,1428240100,1428217847,26
4,5,"[1, 298, 290, 286, 289, 285, 287, 292, 293]",1.425988e+09,1.088851e+13,1427946554,1415933104,9
5,6,"[338, 302, 1, 292]",1.428761e+09,9.504178e+11,1429528852,1427463113,4
...,...,...,...,...,...,...,...
239,240,"[285, 1, 287]",1.416641e+09,1.526822e+12,1419128047,1415944349,3
242,243,"[329, 317, 1, 285]",1.429664e+09,4.192383e+10,1429844825,1429450034,4
243,244,"[286, 298, 308, 327, 292, 295, 306, 285, 287, ...",1.428131e+09,8.906950e+11,1429439514,1426859087,11
244,245,"[285, 286, 291, 1]",1.427662e+09,4.497542e+12,1429980214,1425042775,4


In [2]:
from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = net_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
net_inter['scene'] = predict_y
net_inter

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len,scene
1,2,"[1, 297, 291, 286, 285, 290]",1.420932e+09,2.097459e+13,1425100667,1414155169,6,2
2,3,"[1, 285, 286, 287, 289, 290, 292, 294, 296, 29...",1.425333e+09,1.421498e+13,1426688331,1415095892,12,2
3,4,"[1, 285, 286, 287, 288, 289, 290, 291, 292, 29...",1.428229e+09,5.407589e+07,1428240100,1428217847,26,3
4,5,"[1, 298, 290, 286, 289, 285, 287, 292, 293]",1.425988e+09,1.088851e+13,1427946554,1415933104,9,2
5,6,"[338, 302, 1, 292]",1.428761e+09,9.504178e+11,1429528852,1427463113,4,1
...,...,...,...,...,...,...,...,...
239,240,"[285, 1, 287]",1.416641e+09,1.526822e+12,1419128047,1415944349,3,0
242,243,"[329, 317, 1, 285]",1.429664e+09,4.192383e+10,1429844825,1429450034,4,1
243,244,"[286, 298, 308, 327, 292, 295, 306, 285, 287, ...",1.428131e+09,8.906950e+11,1429439514,1426859087,11,1
244,245,"[285, 286, 291, 1]",1.427662e+09,4.497542e+12,1429980214,1425042775,4,1


In [5]:
beh_count = net.groupby(['user_id','behavior']).count().reset_index()
net_inter['done_count'] = beh_count[beh_count['behavior']=='done']['item_id']
net_inter['watch_count'] = beh_count[beh_count['behavior']=='watch']['item_id']
net_inter['enter_count'] = beh_count[beh_count['behavior']=='enter']['item_id']
net_inter['tag_count'] = beh_count[beh_count['behavior']=='tag']['item_id']
net_inter['dur_count'] = beh_count[beh_count['behavior']=='dur']['item_id']
net_inter = net_inter[net_inter['inter_len']>=3]
net_inter = net_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = net_inter[['done_count','watch_count','enter_count','tag_count','dur_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
net_inter['cat'] = predict_y
net_inter

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len,scene,done_count,watch_count,enter_count,tag_count,dur_count,cat
1,2,"[1, 297, 291, 286, 285, 290]",1.420932e+09,2.097459e+13,1425100667,1414155169,6,2,0.0,0.0,1.0,0.0,0.0,1
2,3,"[1, 285, 286, 287, 289, 290, 292, 294, 296, 29...",1.425333e+09,1.421498e+13,1426688331,1415095892,12,2,0.0,0.0,0.0,1.0,0.0,0
3,4,"[1, 285, 286, 287, 288, 289, 290, 291, 292, 29...",1.428229e+09,5.407589e+07,1428240100,1428217847,26,3,0.0,1.0,0.0,0.0,0.0,2
4,5,"[1, 298, 290, 286, 289, 285, 287, 292, 293]",1.425988e+09,1.088851e+13,1427946554,1415933104,9,2,6.0,0.0,0.0,0.0,0.0,0
5,6,"[338, 302, 1, 292]",1.428761e+09,9.504178e+11,1429528852,1427463113,4,1,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,240,"[285, 1, 287]",1.416641e+09,1.526822e+12,1419128047,1415944349,3,0,9.0,0.0,0.0,0.0,0.0,3
242,243,"[329, 317, 1, 285]",1.429664e+09,4.192383e+10,1429844825,1429450034,4,1,0.0,0.0,0.0,4.0,0.0,0
243,244,"[286, 298, 308, 327, 292, 295, 306, 285, 287, ...",1.428131e+09,8.906950e+11,1429439514,1426859087,11,1,0.0,1.0,0.0,0.0,0.0,2
244,245,"[285, 286, 291, 1]",1.427662e+09,4.497542e+12,1429980214,1425042775,4,1,1.0,0.0,0.0,0.0,0.0,0


In [7]:
alldata = []
for user in net_inter['user_id'].tolist():
    # print(user)
    seq_len = len(net_inter[net_inter['user_id']==user]['item_id'].item())
    total_seq = net_inter[net_inter['user_id']==user]['item_id'].item().tolist()
    scene = net_inter[net_inter['user_id']==user]['scene'].item()
    cat = net_inter[net_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
net_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
net_new

,user_id,inter_seq,scene,cat,target,pred_path
0,2,[1],2,1,297,[297]
1,2,[1],2,1,291,"[297, 291]"
2,2,[1],2,1,286,"[297, 291, 286]"
3,2,[1],2,1,285,"[297, 291, 286, 285]"
4,2,"[1, 297]",2,1,291,[291]
...,...,...,...,...,...,...
12067,245,[285],1,0,291,"[286, 291]"
12068,245,"[285, 286]",1,0,291,[291]
12069,246,[285],1,1,286,[286]
12070,246,[285],1,1,287,"[286, 287]"


In [8]:
net_test = net_new[net_new['pred_path'].apply(lambda x: len(x))<=5].sample(frac=0.25)
net_train = net_new[~net_new.index.isin(net_test)]
net_test

,user_id,inter_seq,scene,cat,target,pred_path
4083,106,"[1, 294, 304, 333, 303, 326, 302, 320, 286, 33...",0,0,293,"[324, 305, 292, 313, 293]"
4161,111,"[286, 294, 295, 290, 330, 334]",1,0,289,[289]
1183,37,[1],0,0,307,"[285, 286, 289, 307]"
10789,213,"[293, 309, 285, 296, 297, 304, 288, 287, 300, ...",1,0,1,"[332, 294, 1]"
517,15,[1],2,3,289,"[297, 293, 301, 289]"
...,...,...,...,...,...,...
11835,220,"[1, 285, 286, 287, 303, 290, 291, 292, 306, 31...",0,0,310,"[338, 310]"
11848,222,[287],0,1,293,"[289, 1, 290, 292, 293]"
994,26,"[288, 298, 291, 294, 1]",0,0,300,"[293, 290, 287, 300]"
4732,115,"[286, 287, 288, 289, 290, 291, 292, 293, 294, ...",3,0,310,"[309, 310]"


In [9]:
net_test.to_csv('net/net_test_same.csv',header=True,sep='\t',index=False)
net_train.to_csv('net/net_train_same.csv',header=True,sep='\t',index=False)

In [11]:
import pandas as pd

os = pd.read_csv('./os/os_mb_KG.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
os_user_count = os.groupby('user_id').count().reset_index()[['user_id','item_id']]
os_inter = os[os['user_id'].isin(os_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()

os_inter['time_avg'] = os.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
os_inter['time_var'] = os.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
os_inter['time_max'] = os.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
os_inter['time_min'] = os.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
os_inter['inter_len'] = os_inter['item_id'].apply(lambda x : len(x))
os_inter = os_inter[os_inter['inter_len']>=3]
os_inter = os_inter.fillna(0)
os_inter

,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len
0,1,"[397, 428, 429, 417]",1.429499e+09,8.711629e+05,1429499699,1429497734,4
2,3,"[397, 400, 30, 60, 119, 109, 43, 427]",1.418265e+09,1.983274e+13,1424977784,1414155568,8
3,4,"[397, 398, 399, 400, 401, 402, 407, 410, 411, ...",1.427878e+09,9.657994e+11,1428730050,1426748094,23
4,5,"[398, 397, 406, 417, 427, 438, 457, 466, 476, ...",1.428817e+09,3.236713e+11,1429933537,1427292523,10
5,6,"[397, 128, 400, 30, 407, 398, 427, 432, 448, 3...",1.424948e+09,1.252034e+13,1427879290,1415932119,18
...,...,...,...,...,...,...,...
292,293,"[397, 405, 417, 427]",1.420773e+09,6.580631e+12,1422189065,1416024172,4
293,294,"[399, 397, 398, 400, 401]",1.428673e+09,1.763792e+10,1428743052,1428420554,5
300,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409, ...",1.427870e+09,8.242692e+11,1429278219,1426769184,12
301,302,"[397, 407, 408]",1.428132e+09,5.652596e+10,1428218521,1427543850,3


In [12]:
from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = os_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
os_inter['scene'] = predict_y
os_inter

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len,scene
0,1,"[397, 428, 429, 417]",1.429499e+09,8.711629e+05,1429499699,1429497734,4,0
2,3,"[397, 400, 30, 60, 119, 109, 43, 427]",1.418265e+09,1.983274e+13,1424977784,1414155568,8,2
3,4,"[397, 398, 399, 400, 401, 402, 407, 410, 411, ...",1.427878e+09,9.657994e+11,1428730050,1426748094,23,0
4,5,"[398, 397, 406, 417, 427, 438, 457, 466, 476, ...",1.428817e+09,3.236713e+11,1429933537,1427292523,10,0
5,6,"[397, 128, 400, 30, 407, 398, 427, 432, 448, 3...",1.424948e+09,1.252034e+13,1427879290,1415932119,18,2
...,...,...,...,...,...,...,...,...
292,293,"[397, 405, 417, 427]",1.420773e+09,6.580631e+12,1422189065,1416024172,4,1
293,294,"[399, 397, 398, 400, 401]",1.428673e+09,1.763792e+10,1428743052,1428420554,5,0
300,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409, ...",1.427870e+09,8.242692e+11,1429278219,1426769184,12,0
301,302,"[397, 407, 408]",1.428132e+09,5.652596e+10,1428218521,1427543850,3,0


In [15]:
beh_count = os.groupby(['user_id','behavior']).count().reset_index()
os_inter['done_count'] = beh_count[beh_count['behavior']=='done']['item_id']
os_inter['watch_count'] = beh_count[beh_count['behavior']=='watch']['item_id']
os_inter['enter_count'] = beh_count[beh_count['behavior']=='enter']['item_id']
os_inter['tag_count'] = beh_count[beh_count['behavior']=='tag']['item_id']
os_inter['dur_count'] = beh_count[beh_count['behavior']=='dur']['item_id']
os_inter = os_inter[os_inter['inter_len']>=3]
os_inter = os_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = os_inter[['done_count','watch_count','enter_count','tag_count','dur_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
os_inter['cat'] = predict_y
os_inter

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,item_id,time_avg,time_var,time_max,time_min,inter_len,scene,done_count,watch_count,enter_count,tag_count,dur_count,cat
0,1,"[397, 428, 429, 417]",1.429499e+09,8.711629e+05,1429499699,1429497734,4,0,3.0,0.0,0.0,0.0,0.0,0
2,3,"[397, 400, 30, 60, 119, 109, 43, 427]",1.418265e+09,1.983274e+13,1424977784,1414155568,8,2,0.0,0.0,0.0,4.0,0.0,0
3,4,"[397, 398, 399, 400, 401, 402, 407, 410, 411, ...",1.427878e+09,9.657994e+11,1428730050,1426748094,23,0,1.0,0.0,0.0,0.0,0.0,0
4,5,"[398, 397, 406, 417, 427, 438, 457, 466, 476, ...",1.428817e+09,3.236713e+11,1429933537,1427292523,10,0,0.0,0.0,1.0,0.0,0.0,1
5,6,"[397, 128, 400, 30, 407, 398, 427, 432, 448, 3...",1.424948e+09,1.252034e+13,1427879290,1415932119,18,2,0.0,0.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,293,"[397, 405, 417, 427]",1.420773e+09,6.580631e+12,1422189065,1416024172,4,1,0.0,0.0,0.0,0.0,0.0,0
293,294,"[399, 397, 398, 400, 401]",1.428673e+09,1.763792e+10,1428743052,1428420554,5,0,0.0,0.0,1.0,0.0,0.0,1
300,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409, ...",1.427870e+09,8.242692e+11,1429278219,1426769184,12,0,1.0,0.0,0.0,0.0,0.0,0
301,302,"[397, 407, 408]",1.428132e+09,5.652596e+10,1428218521,1427543850,3,0,0.0,0.0,1.0,0.0,0.0,1


In [17]:
alldata = []
for user in os_inter['user_id'].tolist():
    # print(user)
    seq_len = len(os_inter[os_inter['user_id']==user]['item_id'].item())
    total_seq = os_inter[os_inter['user_id']==user]['item_id'].item().tolist()
    scene = os_inter[os_inter['user_id']==user]['scene'].item()
    cat = os_inter[os_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
os_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
os_new


,user_id,inter_seq,scene,cat,target,pred_path
0,1,[397],0,0,428,[428]
1,1,[397],0,0,429,"[428, 429]"
2,1,"[397, 428]",0,0,429,[429]
3,3,[397],2,0,400,[400]
4,3,[397],2,0,30,"[400, 30]"
...,...,...,...,...,...,...
23164,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409]",0,0,405,[405]
23165,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409]",0,0,495,"[405, 495]"
23166,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409, ...",0,0,495,[495]
23167,302,[397],0,1,407,[407]


In [22]:
os_test = os_new[os_new['pred_path'].apply(lambda x: len(x))<=5].sample(frac=0.32)
os_train = os_new[~os_new.index.isin(os_test)]
os_test

,user_id,inter_seq,scene,cat,target,pred_path
4601,20,"[398, 399, 400, 402, 401, 403, 404, 406, 405, ...",3,2,451,"[450, 451]"
21656,262,"[397, 398, 399, 401]",0,0,417,"[402, 403, 404, 405, 417]"
19090,223,"[397, 399, 429, 417, 455, 485, 487, 452]",0,2,398,"[486, 488, 470, 468, 398]"
5877,21,"[419, 420, 421, 418, 422, 424, 417, 423, 425, ...",0,0,451,"[450, 451]"
22059,264,"[397, 32, 42, 44, 90, 30, 40, 47, 100, 74, 78,...",1,0,174,"[110, 111, 174]"
...,...,...,...,...,...,...
8667,83,"[397, 38, 40, 90, 45, 48, 49, 47, 50, 60, 44, ...",1,0,29,"[110, 105, 75, 78, 29]"
10265,97,"[16, 38, 397, 115, 128, 400, 6, 21, 29, 30, 31...",1,0,44,"[43, 44]"
18020,207,"[215, 400, 397, 90, 40, 42, 48, 30, 43]",1,0,438,[438]
6263,38,"[397, 16, 29, 21, 32, 400]",1,1,31,"[6, 31]"


In [ ]:
os_test.to_csv('os/os_test_same.csv',header=True,sep='\t',index=False)
os_train.to_csv('os/os_train_same.csv',header=True,sep='\t',index=False)